In [1]:
import os
import io
import os
import dill
import json
import boto3
import torch
import transformers
import torch.nn as nn
from datetime import datetime
from contextlib import contextmanager
from tempfile import NamedTemporaryFile
from torch.utils.data import Dataset, DataLoader

In [2]:
class DataAdapter:
    def prefix_and_not_empty(self, bucket_name: str, prefix: str) -> bool:
        bucket = self.s3_resource.Bucket(bucket_name)
        count = bucket.objects.filter(Prefix=prefix)
        return len(list(count))

    def get_data_from_s3(self, bucket_name, prefix, data_file):
        if self.prefix_and_not_empty(bucket_name=bucket_name, prefix=prefix):
            data_object = self.s3_client.get_object(
                Bucket=bucket_name, Key=os.path.join(prefix, data_file)
            )["Body"]
            return data_object
        else:
            raise RuntimeError(
                f"{data_file} is not present in S3, Try by uploading {data_file} onto S3"
            )

In [3]:
class LabelDataset(Dataset):
    """

    LabelDataset class - to load the dataset used the __getitem__ fashion supported by the Pytorch.
    The loader supports the JSON and the csv format for parsing the input to the network.
        :param mode: mode of Label classifier model (train, eval or serve)
        :param text: input text for train, eval and serve components
        :param label: output label for train and eval components
        :param func_test: True for functional testing of package
        :param tokenizer: tokenizer from huggingface library


    """

    def __init__(self, labels, max_length=512, mode="train"):
        self.mode = mode
        self.labels = labels
        self.max_length = max_length

    def __getitem__(self, item):
        """

        Returns tokenized tensors for text and label(if mode is train or eval) for the given index.
            :param: item: index to fetch the data.
            :returns dict: dictionary of tensors containing input_ids, attention_mask,
                           token_type_ids and label(output if mode is train or eval)

        """
        if self.mode in ["train_eval", "train", "eval"]:
            text = str(self.text[item])
            processed_text = " ".join(text.split())
            label = self.label[item]
            inputs = self.tokenizer.encode_plus(
                processed_text,
                None,
                add_special_tokens=True,
                max_length=self.max_length,
                truncation=True,
            )
            padding_length = self.max_length - len(inputs.get("input_ids"))
            input_ids = inputs.get("input_ids") + ([0] * padding_length)
            attention_mask = inputs.get("attention_mask") + ([0] * padding_length)
            token_type_ids = inputs.get("token_type_ids") + ([0] * padding_length)
            return {
                "input_ids": torch.tensor(input_ids, dtype=torch.long),
                "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
                "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
                "label": torch.tensor(
                    self.labels.index(label.strip()), dtype=torch.long
                ),
            }
        elif self.mode == "serve":
            text = str(self.text[item])
            text = " ".join(text.split())
            inputs = self.tokenizer.encode_plus(
                text,
                None,
                add_special_tokens=True,
                max_length=self.max_length,
                truncation=True,
            )
            padding_length = self.max_length - len(inputs.get("input_ids"))
            input_ids = inputs.get("input_ids") + ([0] * padding_length)
            attention_mask = inputs.get("attention_mask") + ([0] * padding_length)
            token_type_ids = inputs.get("token_type_ids") + ([0] * padding_length)
            return {
                "text": text,
                "input_ids": torch.tensor(input_ids, dtype=torch.long),
                "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
                "token_type_ids": torch.tensor(token_type_ids, dtype=torch.long),
            }

    def __len__(self):
        """

        Returns length of dataset
            :returns int: length of dataset

        """
        return len(self.text)

In [4]:
class LabelBackbone(nn.Module):
    """

    LabelBackbone - Backbone model class for Label Classifier
        :param model: Transformer model name from config.TRANSFORMER_MODEL_LIST or saved transformer model name
        :param tokenizer: tokenizer from huggingface library
        :returns Object of LabelBackbone Model

    """
    def __init__(self, model, tokenizer):
        super(LabelBackbone, self).__init__()
        self.model = model
        self.tokenizer = tokenizer
        

    def forward(self, **kwargs):
        return self.model(**kwargs)
        #             final_output = self.linear(self.drop_out(po))
        return final_output

In [5]:
class ModelAdapter:
    def __init__(self, labels):
        self.labels = labels

    def prefix_exists_and_not_empty(self, bucket_name: str, key: str):

        if not key.endswith("/"):
            key = key + "/"
        resp = self.s3_client.list_objects(
            Bucket=bucket_name, Prefix=key, Delimiter="/", MaxKeys=1
        )
        return "Contents" in resp

    @contextmanager
    def s3_fileobj(self, bucket_name, key):

        """
        Yields a file object from the filename at {bucket}/{key}

        Args:
            bucket (str): Name of the S3 bucket where you model is stored
            key (str): Relative path from the base of your bucket, including the filename and extension of the object to be retrieved.
        """

        obj = self.s3_client.get_object(Bucket=bucket_name, Key=key)
        yield io.BytesIO(obj["Body"].read())

    def get_data_from_s3(self, bucket_name, prefix, data_file):

        data_object = self.s3_client.get_object(
            Bucket=bucket_name, Key=os.path.join(prefix, data_file)
        )["Body"]
        return data_object

    def get_model_tokenizer_from_s3(
        self, bucket_name, prefix, model_name="pytorch_model"
    ):

        if not self.prefix_exists_and_not_empty(
            bucket_name=bucket_name, key=prefix
        ):
            model = transformers.BertForSequenceClassification.from_pretrained(
                "bert-base-uncased", num_labels=len(self.labels)
            )
            tokenizer = transformers.BertTokenizer.from_pretrained(
                "bert-base-uncased"
            )
            self.save_model_tokenizer_into_s3(
                bucket_name=bucket_name,
                prefix=prefix,
                model_name=model_name,
                model=model,
                tokenizer=tokenizer,
            )
            return model, tokenizer

        else:
            tempfile = NamedTemporaryFile()
            with self.s3_fileobj(
                bucket_name=bucket_name, key=f"{prefix}/{model_name}.bin"
            ) as f:
                tempfile.write(f.read())

            with self.s3_fileobj(
                bucket_name=bucket_name, key=f"{prefix}/config.json"
            ) as f:
                dict_data = json.load(f)
                config = transformers.PretrainedConfig.from_dict(dict_data)

            #                 model = PreTrainedModel.from_pretrained(tempfile.name, config=config)
            model = transformers.BertForSequenceClassification.from_pretrained(
                tempfile.name, config=config
            )
            tokenizer = self.get_tokenizer_from_s3(bucket_name, prefix)
            return model, tokenizer

    def get_tokenizer_from_s3(self, bucket_name, prefix):

        tempfile = NamedTemporaryFile()
        with self.s3_fileobj(
            bucket_name=bucket_name, key=f"{prefix}/vocab.txt"
        ) as f:
            tempfile.write(f.read())
        tokenizer = transformers.BertTokenizer.from_pretrained(tempfile.name)
        return tokenizer

    def save_model_tokenizer_into_s3(
        self, bucket_name, prefix, model, model_name="pytorch_model", tokenizer=None
    ):
        buffer = io.BytesIO()
        config_string = model.config.to_json_string()
        self.s3_client.put_object(
            Bucket=bucket_name, Key=f"{prefix}/config.json", Body=config_string
        )
        torch.save(model.state_dict(), buffer)
        self.s3_client.put_object(
            Bucket=bucket_name,
            Key=f"{prefix}/{model_name}.bin",
            Body=buffer.getvalue(),
        )
        if tokenizer:
            vocab_string = ""
            for token, token_id in tokenizer.vocab.items():
                vocab_string = vocab_string + token + "\n"

            vocab_string = vocab_string.strip()
            self.s3_client.put_object(
                Bucket=bucket_name, Key=f"{prefix}/vocab.txt", Body=vocab_string
            )

In [6]:
def train(
    bucket_name: str,
    prefix: str,
    data_file: str,
    labels: list,
    num_epochs: int,
    batch_size: int,
    device: str,
   
):

    """

    Training method for Label Classifier. Saves the model after training of model is completed
        :param num_epochs: Number of epochs for training purpose


    """

    import io
    import os
    import dill
    import json
    import boto3
    import torch
    import numpy as np
    import pandas as pd
    import transformers
    from tqdm import tqdm
    import torch.nn as nn
    from transformers import AdamW
    import torch.nn.functional as f
    from torch.utils.data import DataLoader

    accuracy = []
    buffer = io.BytesIO()
    s3_client = boto3.client("s3")
    s3_resource = boto3.resource("s3")
    LABEL_CLASSIFIER = datetime.now().strftime(
    "FineTunedClassifier_%H-%M-%d-%m-%Y"
)

    
    data_adapter_obj = DataAdapter()
    setattr(data_adapter_obj, "s3_client", s3_client)
    setattr(data_adapter_obj, "s3_resource", s3_resource)
    dataframe = pd.read_csv(
        data_adapter_obj.get_data_from_s3(
            bucket_name=bucket_name, prefix=f"{prefix}/dataset", data_file=data_file
        )
    )
    print("------>dataframe<------", dataframe.head())
    text = dataframe.loc[:, "text"]
    label = dataframe.loc[:, "label"]

    
    model_adapter_obj = ModelAdapter(labels=labels)
    setattr(model_adapter_obj, "s3_client", s3_client)
    pretrained_model, tokenizer = model_adapter_obj.get_model_tokenizer_from_s3(
        bucket_name=bucket_name, prefix=f"{prefix}/models/bert_base_uncased"
    )

    
    dataset_obj = LabelDataset(labels=labels)
    setattr(dataset_obj, "text", text)
    setattr(dataset_obj, "label", label)
    setattr(dataset_obj, "mode", "train")
    setattr(dataset_obj, "tokenizer", tokenizer)
    train_data_loader = DataLoader(dataset_obj, batch_size=batch_size, shuffle=True)

    
    backbone_model = LabelBackbone(model=pretrained_model, tokenizer=tokenizer)

    loss_function = nn.CrossEntropyLoss().to(device)
    optimizer = AdamW(backbone_model.parameters(), lr=3e-5)

    try:
        backbone_model.to(device)
        backbone_model.train()  # Call the train method from the nn.Module base class
        print("Starting the Training Loop ..")  # Training loop start
        for epoch in range(num_epochs):
            train_loss = 0
            train_accuracy = 0
            print(f"[INFO] Epoch {epoch + 1} Started..")
            for index, batch in tqdm(enumerate(train_data_loader)):
                print(
                    f"[INFO] [TRAINING] Epoch {epoch + 1} Iteration {index + 1} Running.."
                )
                optimizer.zero_grad()
                input_ids = batch["input_ids"].to(device)
                attention_mask = batch["attention_mask"].to(device)
                token_type_ids = batch["token_type_ids"].to(device)
                label = batch["label"].to(device).to(device)
                output = backbone_model(
                    input_ids=input_ids, attention_mask=attention_mask, labels=label
                )
                loss = output[0]
                loss.backward()
                optimizer.step()
                train_loss = train_loss + loss.item()
                _, hypothesis = torch.max(output[1], dim=1)
                train_accuracy = (
                    train_accuracy + torch.sum(torch.tensor(hypothesis == label)).item()
                )
            train_accuracy = train_accuracy / (len(train_data_loader) * batch_size)
            accuracy.append(train_accuracy)
            train_loss = train_loss / (len(train_data_loader) * batch_size)
            train_st = f"Training Loss: {train_loss} Train Accuracy: {train_accuracy}"
            print(f"Epoch: {epoch+1} {train_st}")
            model_adapter_obj.save_model_tokenizer_into_s3(
                bucket_name=bucket_name,
                prefix=f"{prefix}/models/{LABEL_CLASSIFIER}",
                model=backbone_model.model,
                tokenizer=tokenizer
            )
        print("Model has been successfully built..")

    except (RuntimeError, MemoryError, ValueError, TypeError) as e:
        print("Training Exception Occurred")
        raise RuntimeError("Training Exception Occurred")

In [ ]:
labels = [
    "Action with Deadline",
    "Announcement",
    "Appreciation",
    "Action",
    "Others",
]
train(bucket_name="hive-s3-lz", prefix="Pratilipi/AI/label-classifier",data_file="label.csv", labels=labels, num_epochs=10, batch_size=8, device="cuda")

------>dataframe<------                                                 text         label
0  Legato takes to the field on 27th and 28th of ...  Announcement
1          Your great work has earned you a vacation  Appreciation
2   I guess Kajari told you to watch those videos...        Action
3  what no confusion for per week 2 weeks scripts...        Others
4           You are full of constructive suggestions  Appreciation


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

Starting the Training Loop ..
[INFO] Epoch 1 Started..
[INFO] [TRAINING] Epoch 1 Iteration 1 Running..


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:103: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
1it [00:00,  1.37it/s]

[INFO] [TRAINING] Epoch 1 Iteration 2 Running..


...

[INFO] [TRAINING] Epoch 10 Iteration 78 Running..


78it [01:01,  1.27it/s]

[INFO] [TRAINING] Epoch 10 Iteration 79 Running..


79it [01:02,  1.26it/s]


Epoch: 10 Training Loss: 0.0022155997119372405 Train Accuracy: 0.9968354430379747
Model has been successfully built..
